# Logistic Regression

This notebook provides an example of how to use SML to read in a dataset, split the data into training and testing data, replace troublesome such as NaNs from the dataset, and perform classifcation on the dataset. For this use-case we use publicly availiable [Spambase Dataset](https://archive.ics.uci.edu/ml/datasets/Spambase) and use naive bayes to classify emails.


## SML Query

### Imports
We Make the nescessary imports to use sml to read in the dataset, split the dataset into training and testing data, replace troublesome values from the dataset and perform classifcation on the dataset.

In [1]:
from sml import execute

Next we create a query statement to `READ` in the data and the file is delimited by ',', the header is not used, and the types of values are numeric and string, next we `SPLIT` the dataset and use 80% of it for training and 20% of it for testing, and lastly, we perform classification using naive bayes on the 58th column, using columns 1-56 as the the predictiors.

In [7]:
query = 'READ "../data/spam.csv" AND \
SPLIT (train = .8, test = 0.2) AND\
CLASSIFY (predictors = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56], label = 58, algorithm = bayes)'

execute(query, verbose=True)


Sml Summary:
   Dataset Path:        ../data/spam.csv
   Delimiter:      None
   Training Set Split:       80.00%
   Testing Set Split:        20.00%
   Predictiors:        ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56']
   Label:         58
   Algorithm:     bayes
   Dataset Preview:
     0     1     2    3     4     5     6     7     8     9  ...    48     49  \
0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00  0.00 ...  0.00  0.000   
1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94 ...  0.00  0.132   
2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25 ...  0.01  0.143   
3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63 ...  0.00  0.137   
4  0.00  0.00  0.00  0.0  0.

## Manually
The subsequent cells below show how the same actions of a SML query can be performed manually.

### Imports
Here we import the necessary libraries needed to perform the same actions as the SML query above.

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn import cross_validation, metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import cross_validation, metrics


### READ

By default the Spambase Dataset does not include it's headers, so we specify it manually, and read that file into a pandas dataframe.

In [4]:
names = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our',
            'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail',
         'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses',
         'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit',
         'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp',
         'word_freq_hpl', 'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs',
         'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
         'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
         'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 'word_freq_re',
         'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'char_freq_;', 'char_freq_(',
         'char_freq_[', 'char_freq_!', 'char_freq_$', 'char_freq_#', 'capital_run_length_average',
         'capital_run_length_longest', 'capital_run_length_total', 'spam']

data = pd.read_csv("../data/Spam.csv", sep="," ,header=None, names=names)
data.head()


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


### SPLIT
We then seperate our labels from our features and use a sklearn function to perform a 80%/20% split our training and testing dataset respectively.

In [5]:
features = data.drop('spam',1)
labels = data['spam']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=.2, random_state=42)


### CLASSIFY

Lastly we fit our naive bayes with our training dataset and make predictions on our testing dataset and display the accuracy.

In [6]:
mNB =  MultinomialNB()
mNB.fit(X_train, y_train)
pred = mNB.predict(X_test)
score = accuracy_score(pred, y_test)

print('Accuracry Score is: %s' % score)

Accuracry Score is: 0.786102062975
